## MOT17 and MOT20 Object Detection finetuning
### This notebook is used to lunch the finetuning of FPN on MOT pedestrian detection benchmark, the code fetches COCO weights for weight initialization

In [ ]:
#select the mot challenge to work with (two are available)
mot_datasets = ["MOT17","MOT20"]
dataset = mot_datasets[0]

In [ ]:
#Use the appropriate path for the selected benchmark
data_path = "../datasets/MOT/%s/train/"%dataset

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()
import csv
# import some common libraries
import numpy as np
import cv2

import random



# import some common detectron2 utilities

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.modeling import build_model
from detectron2.evaluation import COCOEvaluator,PascalVOCDetectionEvaluator
import matplotlib.pyplot as plt
import torch.tensor as tensor
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import inference_on_dataset
import torch
from detectron2.structures.instances import Instances
from detectron2.modeling import build_model
%matplotlib inline

## Dataset parsing

In [ ]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
ARs=[]
AR_cars = []
AR_peds = []
AR_cyc = []
def get_mot_dicts(main_dir):
        
    dataset_dicts = []
    
    items = 0
    car_count=0
    cyc_count = 0
    ped_count =0
    objs = []
    global_image_id = 1
    for folder in os.listdir(main_dir):
        print('processing folder' + folder)
        img_folder = main_dir + folder + '/img1/'
        print(img_folder)
        gt_file = main_dir + folder + '/gt/gt.txt'
        test_img = cv2.imread(img_folder + '000001.jpg')
        
        height, width =test_img.shape[:2]
        print(folder + '%d x %d'%(width,height))
        num_items = len(os.listdir(img_folder))
        for i in range(1,num_items):
            
           
            objs = []
            record = {}
            image_path = img_folder + str(i).zfill(6) + '.jpg'
            record["file_name"] = image_path
            record["image_id"] = global_image_id
            global_image_id+=1
            record["height"] = height
            record["width"] = width
            with open(gt_file, "r") as inf:
                reader = csv.reader(inf, delimiter=',')
                for row in reader:        
                    
                    if int(row[0]) == i and int(round(float(row[6]))) == 1 and int(round(float(row[7]))) == 1 and float(row[8]) >= 0.25:
                        xmin = int(round(float(row[2])))
                        ymin = int(round(float(row[3])))
                        box_width = int(round(float(row[4])))
                        box_height = int(round(float(row[5])))
                        xmax = xmin+ box_width
                        ymax = ymin + box_height
                        
                        

                        obj = {'bbox':[xmin,ymin,xmax,ymax],"category_id": 0,"iscrowd": 0,"bbox_mode": BoxMode.XYXY_ABS}      

                        
                        

                        objs.append(obj)
            record["annotations"] = objs
            dataset_dicts.append(record)
        
          #if(items>100):
            #break
        break
    
    return dataset_dicts



In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train"]:
    DatasetCatalog.register("%s_"%dataset + d, lambda d=d: get_mot_dicts
                            (data_path))
    


## Training Parameters

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os
cfg = get_cfg()
#cfg.merge_from_file("./detectron2_repo/configs/COCO-Detection/faster_rcnn_R_50_C4_3x.yaml")
cfg.merge_from_file("../configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("%s_train"%dataset,)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS="https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl"

#cfg.MODEL.WEIGHTS = "/media/DATA/Users/Issa/Models/MOT17_DET/model_final.pth"
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00015  # pick a good LR
cfg.SOLVER.MAX_ITER = 30000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES=1
  # only has one class (ballon)
cfg.OUTPUT_DIR='../models/MOT/%s_DET'%dataset
print(cfg.OUTPUT_DIR)
print(cfg.MODEL.ANCHOR_GENERATOR)

In [ ]:
#cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS=[[0.25,0.5,1,2]]
#cfg.MODEL.ANCHOR_GENERATOR.SIZES=[[32,64,128,256,512]]
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg,False) 


In [ ]:
trainer.resume_or_load(resume=True)
trainer.train()